In [33]:
#Here we will implement previous code with the DLC package. This requires H5 files...

In [34]:
#Imports

import pandas as pd

import os
import re
import numpy as np
from sympy  import *
import matplotlib.pyplot as plt
import dlc2kinematics
from dlc2kinematics import Visualizer2D

#classes imports
from preprocessing.classes import FilterandThresholds, simpleKinematics

#Initialize classes
splitter = FilterandThresholds('tester')


In [35]:
#Lets see what the dlc2kinematics has to offer

#In this we want to import our files, load the data and extract kinematics
path = r'D:/project_deeplabcut/r&d_cam3_2D-CP-2021-07-19/results_h5/'
configCam3 = r'D:/project_deeplabcut/r&d_cam3_2D-CP-2021-07-19/config.yaml'
files = [file for file in os.listdir(path)]
%matplotlib qt

#Allocate datframes
velocity = []
for file in files:
    df, bodyparts, scorer = dlc2kinematics.load_data(path + file)
    print(file)
    #velocity
    df_vel = dlc2kinematics.compute_velocity(df,bodyparts=['all'], filter_window=3, order=1)
    df_speed = dlc2kinematics.compute_speed(df,bodyparts=['all'], filter_window=3, order=1)

#example for 1 bodypart for velocity:
    df_vel_RBOH = dlc2kinematics.compute_velocity(df,bodyparts=['WristL'],filter_window=3, order=1)
    #df_vel_RBOH.head(3)
    #dlc2kinematics.plot_velocity(df[scorer]['WristL'], df_vel_RBOH)

    viz = Visualizer2D(configCam3, path+file)
    viz.view(show_axes=True, show_grid=True)

print(df_vel)
print(df)
    #To appropriately save this information, we have to build a new dataframe

undistorted_singleedit_10_behavior_shape4_cam3_11090_blind_light_left_success_122421.mp4DLC_resnet50_r&d_cam3_2DJul19shuffle1_400000.h5
undistorted_singleedit_10_behavior_shape4_cam3_21055_blind_light_right_success_121621.mp4DLC_resnet50_r&d_cam3_2DJul19shuffle1_400000.h5
undistorted_singleedit_10_behavior_shape4_cam3_21064_blind_light_left_fail_122221.mp4DLC_resnet50_r&d_cam3_2DJul19shuffle1_400000.h5


IndexError: invalid index to scalar variable.

In [41]:
#Lets try it for camera 2
pathCam2 = r'D:/project_deeplabcut/r&d_cam2_2D-CP-2021-09-14/results_h5/'
configCam2 = r'D:\project_deeplabcut\r&d_cam2_2D-CP-2021-09-14\config.yaml'
files = [file for file in os.listdir(pathCam2)]


#Allocate datframes
velocity = []
for file in files:
    df, bodyparts, scorer = dlc2kinematics.load_data(pathCam2 + file)
    print(file)
    #velocity
    df_vel = dlc2kinematics.compute_velocity(df,bodyparts=['all'], filter_window=3, order=1)
    df_speed = dlc2kinematics.compute_speed(df,bodyparts=['all'], filter_window=3, order=1)

#example for 1 bodypart for velocity:
    df_vel_RBOH = dlc2kinematics.compute_velocity(df,bodyparts=['WristL'],filter_window=3, order=1)
    #df_vel_RBOH.head(3)
    #dlc2kinematics.plot_velocity(df[scorer]['WristL'], df_vel_RBOH)

    viz = Visualizer2D(configCam2, pathCam2+file)
    viz.view(show_axes=True, show_grid=True)

print(df_vel)
print(df)


undistorted_singleedit_1_behavior_shape4_cam2_11090_blind_light_whisk_smell_122421.mp4_1DLC_resnet50_r&d_cam2_2DSep14shuffle1_440000.h5
undistorted_singleedit_1_behavior_shape5_cam2_21066_control_success_light_whisk_smell_122021.mp4DLC_resnet50_r&d_cam2_2DSep14shuffle1_440000.h5
undistorted_singleedit_2_behavior_shape4_cam2_11090_blind_left_122421DLC_resnet50_r&d_cam2_2DSep14shuffle1_440000.h5


IndexError: invalid index to scalar variable.

In [ ]:
#Compute for All bodyparts:
df_vel = dlc2kinematics.compute_velocity(df,bodyparts=['all'], filter_window=3, order=1)
df_speed = dlc2kinematics.compute_speed(df,bodyparts=['all'], filter_window=3, order=1)

#example for 1 bodypart for velocity:
df_vel_RBOH = dlc2kinematics.compute_velocity(df,bodyparts=['WristL'],filter_window=3, order=1)
df_vel_RBOH.head(3)

#plot 1 bodypart, including original DLC positions & new velocity:
%matplotlib inline
import matplotlib.pyplot as plt
dlc2kinematics.plot_velocity(df[scorer]['WristL'], df_vel_RBOH)#, start=42150,end=42450) #42150:42450